In [1]:
import pandas as pd
import numpy as np 
import psycopg2
import cufflinks
from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
connection = psycopg2.connect(dbname='papahere', user='postgres', password='ncr*123')
import plotly.express as px
import plotly.graph_objects as go

In [1]:
candlesize =60
query = f'''
select 
to_timestamp(floor((extract('epoch' from a.exchange_timestamp) / {candlesize} )) * {candlesize}) AT TIME ZONE 'UTC' as candle,
tradingsymbol,
count(*) as nostrades , 
(array_agg (last_price order by a.last_trade_time asc ))[1] o,
(array_agg (last_price order by a.last_trade_time desc ))[1] c,
max (a.last_price) h,
min (a.last_price) l
-- ,max(a.volume_traded)-min(a.volume_traded) volume

from kite_prod.ticks a where a.tradingsymbol = 'BANKNIFTY2230335600PE' and date(exchange_timestamp) = '25feb2022'
-- and extract (hour from a.last_trade_time) between 11 and 12
group by candle , a.tradingsymbol
order by candle 
'''

In [2]:
print(query)


select 
to_timestamp(floor((extract('epoch' from a.exchange_timestamp) / 60 )) * 60) AT TIME ZONE 'UTC' as candle,
tradingsymbol,
count(*) as nostrades , 
(array_agg (last_price order by a.last_trade_time asc ))[1] o,
(array_agg (last_price order by a.last_trade_time desc ))[1] c,
max (a.last_price) h,
min (a.last_price) l
-- ,max(a.volume_traded)-min(a.volume_traded) volume

from kite_prod.ticks a where a.tradingsymbol = 'BANKNIFTY2230335600PE' and date(exchange_timestamp) = '25feb2022'
-- and extract (hour from a.last_trade_time) between 11 and 12
group by candle , a.tradingsymbol
order by candle 



In [112]:
data =pd.read_sql(query, connection)
ha=pd.DataFrame()
# data['hac']=data[['o', 'c', 'h', 'l']].sum(axis=1)/4
# data['hao']=data['o']
# data['hao']=data[['hao', 'hac']].shift(1).sum(axis=1)/2
# data['hah']=data[['hao', 'hac','h']].max(axis=1)
# data['hal']=data[['hao', 'hac','l']].min(axis=1)
# data['green']=data.hac > data.hao
data.to_excel('heiken_ashicheck.xlsx')

C:\Users\ssmgf\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



In [122]:
def hacandles(data):
    dataha=data[[ 'o', 'c', 'h', 'l']].copy()
    for i in range (1,len(dataha)):
        dataha.iat[i,1] = sum(dataha.iloc[i])/4
        dataha.iat[i,0] = sum(dataha.iloc[i-1,:2])/2
        dataha.iat[i,2] = dataha.iloc[i,[0,1,2]].max() 
        dataha.iat[i,-1] = dataha.iloc[i,[0,1,-1]].min() 
        dataha.columns = [ 'hao', 'hac', 'hah', 'hal']
        dataha.index=data.candle
    return dataha



In [123]:
dataha = hacandles(data)

In [124]:
fig = go.Candlestick(x=dataha.index,open=dataha.hao,close= dataha.hac,high=dataha.hah , low=dataha.hal)
fig = go.Figure(data = fig)
fig.layout.xaxis.rangeselector.visible=False
fig.layout.xaxis.rangeslider.visible=False
fig.write_html('check.html')


In [120]:
dataha

,hao,hac,hah,hal
candle,,,,
2022-02-25 10:04:00,542.750000,538.6500,542.75,535.2000
2022-02-25 10:05:00,540.700000,542.6125,552.75,542.6125
2022-02-25 10:06:00,541.656250,539.2125,552.75,542.6125
2022-02-25 10:07:00,540.434375,539.5375,552.75,542.6125
2022-02-25 10:08:00,539.985938,525.5625,552.75,542.6125
...,...,...,...,...
2022-02-25 15:30:00,545.037574,540.0000,552.75,542.6125
2022-02-25 16:26:00,542.518787,540.0000,552.75,542.6125
2022-02-25 16:44:00,541.259394,540.0000,552.75,542.6125


In [111]:
data.index

RangeIndex(start=0, stop=331, step=1)

In [3]:
print(query)


select 
to_timestamp(floor((extract('epoch' from a.exchange_timestamp) / 60 )) * 60) AT TIME ZONE 'UTC' as candle,
tradingsymbol,
count(*) as nostrades , 
(array_agg (last_price order by a.last_trade_time asc ))[1] o,
(array_agg (last_price order by a.last_trade_time desc ))[1] c,
max (a.last_price) h,
min (a.last_price) l
-- ,max(a.volume_traded)-min(a.volume_traded) volume

from kite_prod.ticks a where a.tradingsymbol = 'BANKNIFTY2230335600PE' and date(exchange_timestamp) = '25feb2022'
-- and extract (hour from a.last_trade_time) between 11 and 12
group by candle , a.tradingsymbol
order by candle 

